## Make Profile summaries using Jinja2 and Python Modele

- Fetch SD file from IG
- Transform to Python model
- use Jinja2 template to create a summary markdown file
- save markdown file



### import python modules including R4 fhirclient models

In [1]:
from fhirclient.r4models import structuredefinition as SD
from fhirclient.r4models import narrative as N
from fhirclient.r4models import valueset as VS
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
import fhirclient.r4models.fhirreference as FR
from json import dumps, loads, load
from pprint import pprint
from jinja2 import Environment, FileSystemLoader, select_autoescape
from commonmark import commonmark
from IPython.display import display, HTML, Markdown
import title_map as tm
import os

### Get file and return as dict and generate SD model instance

In [2]:
#in_path = '/Users/ehaas/Documents/FHIR/US-Core-R4/output/StructureDefinition-'
#f_name = 'us-core-patient'
in_path = '/Users/ehaas/.fhir/packages/hl7.fhir.us.core.r4#dev/package'
files = [x for x in os.listdir(in_path) if x.startswith("StructureDefinition")]
files

['StructureDefinition-us-core-careteam.json',
 'StructureDefinition-us-core-medicationstatement.json',
 'StructureDefinition-us-core-direct.json',
 'StructureDefinition-us-core-profile-link.json',
 'StructureDefinition-us-core-device.json',
 'StructureDefinition-us-core-practitioner.json',
 'StructureDefinition-us-core-patient.json',
 'StructureDefinition-us-core-immunization.json',
 'StructureDefinition-us-core-smokingstatus.json',
 'StructureDefinition-us-core-documentreference.json',
 'StructureDefinition-new-us-core-diagnosticreport.json',
 'StructureDefinition-us-core-race.json',
 'StructureDefinition-us-core-medicationrequest.json',
 'StructureDefinition-us-core-diagnosticreport.json',
 'StructureDefinition-us-core-condition.json',
 'StructureDefinition-us-core-birthsex.json',
 'StructureDefinition-us-core-encounter.json',
 'StructureDefinition-us-core-organization.json',
 'StructureDefinition-us-core-pediatric-weight.json',
 'StructureDefinition-us-core-procedure.json',
 'Struct

In [3]:
def open_file(in_path, f_name): # get files
    with open(f'{in_path}/{f_name}') as f:
        r = f.read()
        return(loads(r))
   

In [4]:
 
def expand(vs):
    print(vs)
    fhir_test_server = 'http://fhirtest.uhn.ca/baseDstu3'
    fhir_test_server = 'http://test.fhir.org/r4'

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }

    # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
 
    params = dict(
          url = vs.split('|')[0],
          count = 20,
            )
    
    r = get(f'{fhir_test_server}/ValueSet/$expand', params = params, headers = headers )
    # return r.status_code
    # view  output

    
    if r.status_code == 200:
        print(r.status_code)
        return VS.ValueSet(r.json())

In [5]:
summ_elements = [
    'label',
    'short',
    'min',
    'max',
    'type',
    'binding',
]

for i in files:
    if 'patient' in i:
        sd_dict = open_file(in_path,i)
        sd = SD.StructureDefinition(sd_dict)
        for i in sd.differential.element:
            path = i.path
            print(path)
            for k in summ_elements:
                print(f'differential = {path}.{k} = {getattr(i,k)}')
                if getattr(i,k) == None:
                    snap_element = (s for s in sd.snapshot.element if s.path == path)
                    new_val = getattr(next(snap_element),k)
                    print(f'snapshot = {path}.{k} = {new_val}')
                    setattr(i,k,new_val)
                print(f'differential post if = {path}.{k} = {getattr(i,k)}')

Patient
differential = Patient.label = None
snapshot = Patient.label = None
differential post if = Patient.label = None
differential = Patient.short = None
snapshot = Patient.short = Information about an individual or animal receiving health care services
differential post if = Patient.short = Information about an individual or animal receiving health care services
differential = Patient.min = None
snapshot = Patient.min = 0
differential post if = Patient.min = 0
differential = Patient.max = None
snapshot = Patient.max = *
differential post if = Patient.max = *
differential = Patient.type = None
snapshot = Patient.type = None
differential post if = Patient.type = None
differential = Patient.binding = None
snapshot = Patient.binding = None
differential post if = Patient.binding = None
Patient.extension
differential = Patient.extension.label = None
snapshot = Patient.extension.label = None
differential post if = Patient.extension.label = None
differential = Patient.extension.short = None

### Using Jinja2 Template create md file for summary view

In [9]:
in_path = ''
template_path = 'summary-template.j2'
core_path = 'http://hl7.org/fhir/R4/'

bindings = dict(
    required = f'{core_path}terminologies.html#required',
    extensible = f'{core_path}terminologies.html#extensible',
    preferred =f'{core_path}terminologies.html#preferred',
    example = f'{core_path}terminologies.html#example',
)

def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

env.filters['markdown'] = markdown

template = env.get_template(template_path)
d = template.render(elements = sd.differential.element, title_map=tm.title_map, bindings=bindings,)

print(f'file_name = {sd.id}-summary.md')
display(Markdown(d))

file_name = us-core-patient-summary.md


## Sample Summary Markdown Template...

### Summary of the Mandatory Requirements



1. One or more Identifiers  in `Patient.identifier`

    - which must have an  uri value  in `Patient.identifier.system`

    - which must have a  string value  in `Patient.identifier.value`


1. One or more HumanNames  in `Patient.name`

    - which must have a  string value  in `Patient.name.family`

    - which must have one or more string values  in `Patient.name.given`


1.  A  code  in `Patient.gender`
with a [required](http://hl7.org/fhir/R4/terminologies.html#required)
 binding to [AdministrativeGender](http://hl7.org/fhir/ValueSet/administrative-gender)

### Summary of the Must Support Requirements



1.  A [Race]([&#39;http://hl7.org/fhir/us/core/StructureDefinition/us-core-race&#39;]) Extension  in `Patient.extension`


1.  An [Ethnicity]([&#39;http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity&#39;]) Extension  in `Patient.extension`


1.  A [Birthsex]([&#39;http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex&#39;]) Extension  in `Patient.extension`
with a [required](http://hl7.org/fhir/R4/terminologies.html#required)
 binding to [Birth Sex](http://hl7.org/fhir/us/core/ValueSet/birthsex)

1. One or more ContactPoints  in `Patient.telecom`

   - which must have a  code value  in `Patient.telecom.system`
with a [required](http://hl7.org/fhir/R4/terminologies.html#required)
 binding to [ContactPointSystem](http://hl7.org/fhir/ValueSet/contact-point-system)
   - which must have a  string value  in `Patient.telecom.value`


1.  A  date  in `Patient.birthDate`


1. One or more Addresses  in `Patient.address`

   - which should have one or more string values  in `Patient.address.line`

   - which should have a  string value  in `Patient.address.city`

   - which should have a  string value  in `Patient.address.state`
with a [extensible](http://hl7.org/fhir/R4/terminologies.html#extensible)
 binding to [USPS Two Letter Alphabetic Codes](http://hl7.org/fhir/us/core/ValueSet/us-core-usps-state)
   - which should have a  string value  in `Patient.address.postalCode`


1. One or more BackboneElements  in `Patient.communication`

   - which must have a  CodeableConcept value  in `Patient.communication.language`
with a [extensible](http://hl7.org/fhir/R4/terminologies.html#extensible)\+ [MaxValueSet](general-guidance.html#max-binding)
 binding to [Language codes with language and optionally a region modifier](http://hl7.org/fhir/us/core/ValueSet/simple-language)

### write to file

In [10]:
out_path = ''
out_file = 'search.md'

def write_file(path, f_name, data): # write file
    with open(f'{path}{f_name}', 'w') as f:
        f.write(data)
        
f_name = f'{sd.id}-summary.md'
write_file(out_path,f_name,d)